SSL 证书导入

In [ ]:
import pip_system_certs.wrapt_requests

建立 LLM 会话  
在报告页找免费 api_key 需要根据模型来源初始化会话

In [ ]:
import json
with open(r".\model_map.json", "r", encoding="utf8") as reader:
    model_map = json.load(reader)
    model_avails = {mn:k for k in model_map.keys() for mn in model_map[k]["model_list"]}

print("可选模型 (选一个填到 model_name):")
for mak in model_avails.keys(): print(mak)

In [ ]:
model_name = "gemini-2.5-flash"

In [ ]:
assert model_name in model_avails, "check model_name"
frame = model_avails[model_name]
params = {k: v for k, v in model_map[frame].items() if k not in ["model_list"]}

if frame == "google":
    from google import genai
    client = genai.Client(**params)
elif frame == "openrouter":
    from openai import OpenAI
    client = OpenAI(**params)
else: raise NotImplementedError

认证 github token

In [ ]:
from github.Repository import Repository, NotSet
from github.PullRequest import PullRequest

import github

with open(r".\github_token", "r", encoding="utf8") as reader:
    access_token = reader.readline()

auth = github.Auth.Token(access_token)

筛选 PR

In [ ]:
from typing import Literal, List

def filtre(
        repo:Repository,
        pr_state:Literal["open", "close", "all"]=NotSet, # pr 状态
        sort:Literal["created", "updated", "pushed", "popularity"]=NotSet, # 结果排序方式
        direction:Literal["asc", "desc"]=NotSet, # 结果顺序
        base:str=NotSet, # 目标分支
        head:str=NotSet, # 来源分支/来源用户
        prid:int=None, # 指定 pr 编号
        author:str=None, # 指定 pr 发起人
        day_back:int=None, # 回溯指定天数内
    ) -> List[PullRequest]:
    results = repo.get_pulls(pr_state, sort, direction, base, head)
    if prid: results = [pr for pr in results if pr.number == prid]
    if author: results = [pr for pr in results if pr.user.login == author]
    if day_back:
        # datetime 范围内
        from datetime import datetime, timedelta, timezone
        now = datetime.now().replace(tzinfo=timezone.utc)
        start_from = now - timedelta(days=day_back) 
        results = [pr for pr in results if pr.created_at >= start_from]
    return results

In [ ]:
## https://github.com/{browser-use/browser-use}/pull/xxx.diff
repository = "browser-use/browser-use"

with github.Github(auth=auth) as g:
    repo = g.get_repo(repository)
    prs = filtre(repo, "open", day_back=2) ## 假设在内部，则用 author + day_back 可以查到指定用户的 PR 这里随便选一些。。
    if not prs: raise ValueError("No matching PR found")

prompt 模板 需要调整

In [ ]:
# user_prompt = """Please briefly summarize the changes made to the code from the following diff:
# """
user_prompt = """请简短总结以下的 PR 描述和代码差异，并用数字编号输出差异条目：
PR 描述：
{desc}
代码差异：
{diffs}
"""

In [ ]:
from utils import file_io_utils as fiu
from tqdm.notebook import tqdm

def get_llm_resp(query:str):
    if frame == "google":
        return client.models.generate_content(
            model=model_name,
            contents=[query]
        ).text
    elif frame == "openrouter":
        
        return client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": query}],
            max_tokens=500
        ).choices[0].message.content
    else: raise NotImplementedError

def get_llm_diff_resp(pr):
    diffs = fiu.get_remote_contents(pr.diff_url)
    return get_llm_resp(user_prompt.format(desc=f"{pr.title}\n{pr.body}", diffs=diffs))
    
responses = [get_llm_diff_resp(pr) for pr in tqdm(prs)]

汇总成简报

In [ ]:
summary_prompt = """请根据以下代码差异描述，总结出一份简报，要求根据日期排序："""
for i, resp in enumerate(responses[::-1]): # 从旧到新
    summary_prompt += f"\n\n{prs[::-1][i].created_at}:\n\n{resp}\n"

In [ ]:
response = get_llm_resp(summary_prompt)

In [ ]:
from IPython.display import Markdown, display
display(Markdown(response))

对自己的 pr 添加评论 模拟发布汇报

In [ ]:
with github.Github(auth=auth) as g:
    repo = g.get_repo("icm162/pgat")
    pr = repo.get_pull(1)
    pr.create_issue_comment(response)